In [3]:
%reload_ext autoreload
import os
curr_dir = "/home/mushaikh/private/PointCloudPosePrediction"
os.chdir(curr_dir)
# os.chdir("/Users/mustafa/Documents/grad school/spring 2024/ECE285 - Visual Learning/PointCloudPosePrediction")
import importlib
import time
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision.transforms as transforms
import open3d as o3d
from scipy.io import loadmat
from scipy.spatial.transform import Rotation as R
import tqdm
from typing import Any, Callable, Optional, Tuple
import yaml
import cv2
import matplotlib.pyplot as plt
import core.pointnet as pnet
import core.deep_pose_estimator as dpe
import utils.dataloader
import utils.utils
import utils.vision
importlib.reload(dpe)
importlib.reload(pnet)
importlib.reload(utils.utils)
importlib.reload(utils.vision)
importlib.reload(utils.dataloader)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


<module 'utils.dataloader' from '/home/mushaikh/private/PointCloudPosePrediction/utils/dataloader.py'>

In [5]:
USE_GPU = True

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
    
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
else:
    torch.manual_seed(42)

In [10]:
# load params for current model run; choose model run below after creating yaml file for it
with open("./models/run_1.yaml") as f:
    params = yaml.safe_load(f)

In [11]:
# create data loader for Eden data - outputs pairs of point clouds and relative poses
batch_size = 10
# "/home/mushaikh/private/sample"
dataset=utils.dataloader.EdenDataset("/home/mushaikh/private/sample",
                                            train=True,
                                            keypt_method=params['keypoint_detection']['detection_method'],
                                            pcd_num_pts = params['keypoint_detection']['pcd_num_pts']
                                            )
edenLoader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=1)

In [12]:
# training loop

model = dpe.DeepPoseEstimator(num_pts=params['keypoint_detection']['pcd_num_pts'])
model.to(device)
lr = params['train']['lr']
optimizer = optim.Adam(model.parameters(), lr=lr, betas=(0.9, 0.999))
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)

num_batch = len(dataset) / batch_size

for epoch in range(params['train']['epochs']):
    for i, data in enumerate(edenLoader, 0):
        
        # prepare data
        pcd1, pcd2, quat, translation = data["pcd1"], data["pcd2"], data["quat"], data["translation"]
        pcd1, pcd2 = pcd1.permute(0,2,1), pcd2.permute(0,2,1)
        target = torch.cat((translation.transpose(1,0), quat.transpose(1,0))).transpose(1,0)
        
        # move to gpu
        pcd1, pcd2, quat, translation, target = pcd1.to(device), pcd2.to(device), quat.to(device), translation.to(device), target.to(device)
        
        # reset gradients
        optimizer.zero_grad()
        
        # set mode to train - sets batchnorm/dropout policy etc.
        model.train()
        
        # get preds from model
        pred, trans_feat_1, trans_feat_2 = model(pcd1, pcd2) # predicts 7-length vector of translation/quaternion
        
        # compute loss
        loss = dpe.PoseLoss(pred, target) + \
            pnet.feature_transform_regularizer(trans_feat_1) * 0.001 + \
                pnet.feature_transform_regularizer(trans_feat_1) * 0.001
        
        # backward pass
        loss.backward()
        
        # gradient descent step 
        optimizer.step()
        
#         print("Prediction: ", pred, "\n", "Target: ", target, "\n", "Loss: ", loss)
        if i%10==0:
            print('[%d: %d/%d] train loss: %f' % (epoch, i, num_batch, loss.item()))
        
    # update the learning rate scheduler
    scheduler.step()
    
    # save the model after every epoch
    torch.save(model.state_dict(), '%s/run_1_deep_pose_%d.pth' % (os.path.join(curr_dir,"models"), epoch))

[0: 0/150] train loss: 12.515007
[0: 10/150] train loss: 27.810057


KeyboardInterrupt: 